In [50]:
import sys

sys.path.append(os.path.abspath("../"))
print(sys.path)

['C:\\Users\\junjie.tang\\.conda\\envs\\exceldemo\\python312.zip', 'C:\\Users\\junjie.tang\\.conda\\envs\\exceldemo\\DLLs', 'C:\\Users\\junjie.tang\\.conda\\envs\\exceldemo\\Lib', 'C:\\Users\\junjie.tang\\.conda\\envs\\exceldemo', '', 'C:\\Users\\junjie.tang\\AppData\\Roaming\\Python\\Python312\\site-packages', 'C:\\Users\\junjie.tang\\.conda\\envs\\exceldemo\\Lib\\site-packages', 'C:\\Users\\junjie.tang\\.conda\\envs\\exceldemo\\Lib\\site-packages\\win32', 'C:\\Users\\junjie.tang\\.conda\\envs\\exceldemo\\Lib\\site-packages\\win32\\lib', 'C:\\Users\\junjie.tang\\.conda\\envs\\exceldemo\\Lib\\site-packages\\Pythonwin', 'D:\\PycharmProjects\\exceldemo2\\src', 'D:\\PycharmProjects\\exceldemo2\\src', 'D:\\PycharmProjects\\exceldemo2\\src', 'D:\\PycharmProjects\\exceldemo2', 'D:\\PycharmProjects\\exceldemo2']


In [51]:
from src.prompt_manager import PromptManager
from src.config import Config
from pydantic import BaseModel
from src.llm_singleton import OpenAIClientSingleton
from src.token_singleton import TokenizerSingleton
from src.extract_item import item
from src.utils import build_messages
import json
config = Config()
render_parameters = item().render_parameters
for key, value in config.model_dump().items():
    PromptManager().register_base(key,value)
    if key in render_parameters:
        PromptManager().render_base(key,**render_parameters.get(key))
    else:
        PromptManager().render_base(key)

In [41]:
print(PromptManager().get("type_system"))

#  角色说明
- 你是一个智能助手，你将收到一个Json格式的Excel表格，你需要识别出这个表格的结构。

---

#  输入格式说明
- 用户输入的整个 JSON 是一个对象（`Object`），表示一张 Excel 表格。
- 每个键的格式为 `"行号X"`，表示 Excel 中的第 X 行（如 `"行号0"` 表示第一行）。
- 每一行的值是一个嵌套对象，对应该行中的若干单元格数据。
  - 该对象的键为 `"列号Y"`，表示 Excel 中的第 Y 列（如 `"列号0"` 表示第一列）。
  - 该键对应的值为该单元格中的文本内容。
- 如果某个单元格为空，在该行的对象中会省略对应的 `"列号Y"`。
- 因此，整个 JSON 实际上是以“行号 + 列号”为二维坐标的方式来表示 Excel 表格的内容。
- 示例：
```json
{
  "行号0": {
    "列号0": "购电月份",
    "列号1": "总电量",
    "列号2": "发电客户编号"
  },
  "行号1": {
    "列号0": "2024年12月",
    "列号2": "15000"
  }
}
···

---

#  Excel表格结构说明
- 结构化表格：这种表格的开头是几行表头，剩下的行全是数据行或者合计行。
- 半结构化表格：这种表格主要上是由几个子表格组成。

---

#  任务要求
- 识别出表格所属类别。输出1代表结构化表格，2代表半结构化表格。

---

#  输出格式（JSON）（严格遵循）:
```json
{
  "type": enum[1,2]
}
```

---




In [42]:
input = {
  "行号0": {
    "列号0": "购电月份",
    "列号1": "总电量",
    "列号2": "发电客户编号"
  },
  "行号1": {
    "列号0": "2024年12月",
    "列号2": "15000"
  }
}




In [43]:
messages = build_messages('你好','用户输入',input)

In [44]:
print(messages)

[{'role': 'system', 'content': '你好'}, {'role': 'user', 'content': "用户输入\n{'行号0': {'列号0': '购电月份', '列号1': '总电量', '列号2': '发电客户编号'}, '行号1': {'列号0': '2024年12月', '列号2': '15000'}}/no_think"}]


In [52]:
tokenizer = TokenizerSingleton().get_tokenizer()


In [53]:
text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

In [54]:
print(text)

<|im_start|>system
你好<|im_end|>
<|im_start|>user
用户输入
{'行号0': {'列号0': '购电月份', '列号1': '总电量', '列号2': '发电客户编号'}, '行号1': {'列号0': '2024年12月', '列号2': '15000'}}/no_think<|im_end|>
<|im_start|>assistant



In [55]:
import json

# data = {"name": "Alice", "age": 30, "city": "Beijing"}

# # 使用 f-string 输出为 JSON 格式
# print(f"{json.dumps(data, ensure_ascii=False)}")


In [60]:
print(len(tokenizer(text)["input_ids"]))

87


In [92]:
import json
input2 = json.dumps(input, separators=(",", ":"),ensure_ascii=False,indent=1)

In [93]:
print(input2
     )

{
 "行号0": {
  "列号0": "购电月份",
  "列号1": "总电量",
  "列号2": "发电客户编号"
 },
 "行号1": {
  "列号0": "2024年12月",
  "列号2": "15000"
 }
}


In [89]:
messages2 = build_messages('你好','用户输入',input2)
text2 = tokenizer.apply_chat_template(messages2, tokenize=False, add_generation_prompt=True)
print(text2)

<|im_start|>system
你好<|im_end|>
<|im_start|>user
用户输入
{
 "行号0": {
  "列号0": "购电月份",
  "列号1": "总电量",
  "列号2": "发电客户编号"
 },
 "行号1": {
  "列号0": "2024年12月",
  "列号2": "15000"
 }
}/no_think<|im_end|>
<|im_start|>assistant



In [90]:
print(len(tokenizer(text2)["input_ids"]))

97


In [94]:
print(PromptManager().get("depart_system"))

#  角色说明
- 你是一个智能助手，用户将提供一个 Json 格式的 Excel 表格。你的任务是将这个Excel划分成一个个独立的子表格。以标准 JSON 格式返回结果。
  
---

#  输入格式说明
- 用户输入的整个 JSON 是一个对象（`Object`），表示一张 Excel 表格。
- 每个键的格式为 `"行号X"`，表示 Excel 中的第 X 行（如 `"行号0"` 表示第一行）。
- 每一行的值是一个嵌套对象，对应该行中的若干单元格数据。
  - 该对象的键为 `"列号Y"`，表示 Excel 中的第 Y 列（如 `"列号0"` 表示第一列）。
  - 该键对应的值为该单元格中的文本内容。
- 如果某个单元格为空，在该行的对象中会省略对应的 `"列号Y"`。
- 因此，整个 JSON 实际上是以“行号 + 列号”为二维坐标的方式来表示 Excel 表格的内容。
- 示例：
```json
{
  "行号0": {
    "列号0": "购电月份",
    "列号1": "总电量",
    "列号2": "发电客户编号"
  },
  "行号1": {
    "列号0": "2024年12月",
    "列号2": "15000"
  }
}
```


---

#  任务说明
- 你将收到一个以JSON格式表示的Excel文件，其中包含多个结构相似的子表格块和一些结构不确定的其他子表格构成。请拆分这些子表格块。
- 确保每个子表格块都被单独提取出来。每个子表格块由该子表格第一行行号和最后一行行号构成。
- 用列表[起始行号, 结束行号]表示子表格范围。
- 识别出所有结构相似的子表格，如果这类子表格内容有所缺失，仍然算作同类子表格。
- 识别出表格中结构不确定的其他子表格。

---

# 注意事项
- 应该做到让excel中的每行都被划分到某个子表格范围之内，不要丢失任意一行。

---

#  输出要求
- 输出必须是一个 **JSON 对象**，包含两个字段：`main` 和 `extra`。
- 每个字段的值是一个二维列表，列表中的每个元素是一个子表格块的行号范围，格式为 `[起始行号, 结束行号]`。
- `main` 表示结构相同的子表格范围集合。
- `extra` 表示结构不确定子表格范围集合。
- 子表格范围必

In [95]:
print(PromptManager().get("get_head_system"))

#  角色说明
- 你是一个智能表格助手。用户将提供一个 Json 格式的 Excel 表格。你的任务是从这个表格中识别表头的最后一行是哪一行，并以标准 JSON 格式返回结果。

---
   
#  输入格式说明
- 用户输入的整个 JSON 是一个对象（`Object`），表示一张 Excel 表格。
- 每个键的格式为 `"行号X"`，表示 Excel 中的第 X 行（如 `"行号0"` 表示第一行）。
- 每一行的值是一个嵌套对象，对应该行中的若干单元格数据。
  - 该对象的键为 `"列号Y"`，表示 Excel 中的第 Y 列（如 `"列号0"` 表示第一列）。
  - 该键对应的值为该单元格中的文本内容。
- 如果某个单元格为空，在该行的对象中会省略对应的 `"列号Y"`。
- 因此，整个 JSON 实际上是以“行号 + 列号”为二维坐标的方式来表示 Excel 表格的内容。
- 示例：
```json
{
  "行号0": {
    "列号0": "购电月份",
    "列号1": "总电量",
    "列号2": "发电客户编号"
  },
  "行号1": {
    "列号0": "2024年12月",
    "列号2": "15000"
  }
}

---

#  任务描述
- 你需要识别出开头哪几行是表头行,从哪一行开始是数据行,找到表头和数据行的分界点,也就是说从表头最后一行开始,下一行全是数据行。
- 输出表头最后一行的行号,注意行号用数字表示(比如用`1`表示`行号1`)。

---

#  输出格式（严格遵循）
- 示例：
```json
{
  "index": 9
}
```

---




In [96]:
print(PromptManager().get("get_index_system"))

#  角色说明
你是一个智能表格助手。用户将提供一个 Json 格式的 Excel 表格。你的任务是识别出电费结算单单中各个字段所在的列号。以标准 JSON 格式返回结果。

---

#  输入格式说明
- 用户输入的整个 JSON 是一个对象（`Object`），表示一张 Excel 表格。
- 每个键的格式为 `"行号X"`，表示 Excel 中的第 X 行（如 `"行号0"` 表示第一行）。
- 每一行的值是一个嵌套对象，对应该行中的若干单元格数据。
  - 该对象的键为 `"列号Y"`，表示 Excel 中的第 Y 列（如 `"列号0"` 表示第一列）。
  - 该键对应的值为该单元格中的文本内容。
- 如果某个单元格为空，在该行的对象中会省略对应的 `"列号Y"`。
- 因此，整个 JSON 实际上是以“行号 + 列号”为二维坐标的方式来表示 Excel 表格的内容。
- 示例：
```json
{
  "行号0": {
    "列号0": "购电月份",
    "列号1": "总电量",
    "列号2": "发电客户编号"
  },
  "行号1": {
    "列号0": "2024年12月",
    "列号2": "15000"
  }
}
```

---

#  电费结算单说明
- 描述：电费结算单是供电局向供电公司采购电力后，所开具的财务支付凭证，作为核算、结算与财务管理的重要依据。

---

#  电费结算单各个字段说明（共 11 个）

## 1. `start_time`
- 描述：已付电费的起始时间，即这次已缴费用所覆盖周期的开始月份。
- 匹配关键词：`年月`、`电费年月`、`应付年月`、`购电月份`。

## 2. `end_time`
- 描述：已付电费的起始时间，即这次已缴费用所覆盖周期的结束月份。
- 匹配关键词：`年月`、`电费年月`、`应付年月`、`购电月份`。

## 3. `total_electricity`
- 描述：总发电量。
- 匹配关键词：`总电量`、`发电量`。
 **值类型**：数字或者数字字符串。

## 4. `grid_electricity`
- 描述：总发电量中并入电网的部分电量。
- 匹配关键词：`上网电量`、`总上网电量`。
- **值类型**：数字或者数字字符串。

## 

In [98]:
print(PromptManager().get("get_head_common_system"))

#  角色说明
- 你是一个智能表格助手。用户将提供一个 Json 格式的 Excel 表头。你的任务是从这些表头文本中识别并提取电费结算单中的字段值，并以标准 JSON 格式返回结果。

---

#  输入格式说明
- 用户输入的整个 JSON 是一个对象（`Object`），表示一张 Excel 表格。
- 每个键的格式为 `"行号X"`，表示 Excel 中的第 X 行（如 `"行号0"` 表示第一行）。
- 每一行的值是一个嵌套对象，对应该行中的若干单元格数据。
  - 该对象的键为 `"列号Y"`，表示 Excel 中的第 Y 列（如 `"列号0"` 表示第一列）。
  - 该键对应的值为该单元格中的文本内容。
- 如果某个单元格为空，在该行的对象中会省略对应的 `"列号Y"`。
- 因此，整个 JSON 实际上是以“行号 + 列号”为二维坐标的方式来表示 Excel 表格的内容。
- 示例：
```json

{
  "行号0": {
    "列号0": "购电月份",
    "列号1": "总电量",
    "列号2": "发电客户编号"
  },
  "行号1": {
    "列号0": "2024年12月",
    "列号2": "15000"
  }
}
```

---

#  电费结算单说明
- 描述：电费结算单是供电局向供电公司采购电力后，所开具的财务支付凭证，作为核算、结算与财务管理的重要依据。

---

#  电费结算单各个字段说明（共 11 个）

## 1. `start_time`
- 描述：已付电费的起始时间，即这次已缴费用所覆盖周期的开始月份。
- 匹配关键词：`年月`、`电费年月`、`应付年月`、`购电月份`。

## 2. `end_time`
- 描述：已付电费的起始时间，即这次已缴费用所覆盖周期的结束月份。
- 匹配关键词：`年月`、`电费年月`、`应付年月`、`购电月份`。

## 3. `total_electricity`
- 描述：总发电量。
- 匹配关键词：`总电量`、`发电量`。
 **值类型**：数字或者数字字符串。

## 4. `grid_electricity`
- 描述：总发电量中并入电网的部分电量。
- 匹配关键词：`上网电量`、`总上网电量`。
- **值类型**：数字或者数字字